In [1]:
import ray
import ray.rllib.algorithms.ppo as ppo
from ray.tune.logger import pretty_print
from ray.tune.registry import register_env
from ray.air import RunConfig
from ray import air, tune

import sys

import gym
import numpy as np
import scipy.integrate as sc_integrate

import symbtools as st
import sympy as sp
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt

## Mathematical system description with SymPy / symbtools

In [2]:
# load model etc. from pickle of flatness analysis notebook
with open("double_crane_model.pcl", "rb") as pfile:
    data = pickle.load(pfile)
    locals().update(data)

In [3]:
mod.xx

Matrix([
[   p1],
[   p2],
[   p3],
[   q1],
[   q2],
[pdot1],
[pdot2],
[pdot3],
[qdot1],
[qdot2]])

In [4]:
mod.calc_state_eq(force_recalculation=True)
mod.eqns

Matrix([
[                                                                                                                                                                                                                  m2*pddot1 - tau3*(p1 - q1 - s2*cos(p3))/sqrt((p2 - s2*sin(p3))**2 + (p1 - q1 - s2*cos(p3))**2) - tau4*(-l0 + p1 - q2 + s2*cos(p3))/sqrt((p2 + s2*sin(p3))**2 + (-l0 + p1 - q2 + s2*cos(p3))**2)],
[                                                                                                                                                                                                                           g*m2 + m2*pddot2 - tau3*(p2 - s2*sin(p3))/sqrt((p2 - s2*sin(p3))**2 + (p1 - q1 - s2*cos(p3))**2) - tau4*(p2 + s2*sin(p3))/sqrt((p2 + s2*sin(p3))**2 + (-l0 + p1 - q2 + s2*cos(p3))**2)],
[J2*pddot3 + s2*tau3*(p2 - s2*sin(p3))*cos(p3)/sqrt((p2 - s2*sin(p3))**2 + (p1 - q1 - s2*cos(p3))**2) - s2*tau3*(p1 - q1 - s2*cos(p3))*sin(p3)/sqrt((p2 - s2*sin(p3))**2 + (p1 - q1 -

## Gym environment

In [5]:
from abc import abstractmethod
import inspect
import math
from typing import Optional, Union

import numpy as np
from scipy.integrate import solve_ivp

import gym
from gym import logger, spaces
#from gym.envs.classic_control import utils
from gym.error import DependencyNotInstalled
#from gym.utils.renderer import Renderer

class StateSpaceModel(gym.Env):
    """ Environment subclass that uses a state space model of the form dx/dt = f(x, u)
    to represent the environments dynamics.

    Args:
        ode
        cost
        x0
        uDim

    Attributes:
        ode (function): ODE for simulation
        cost (function): cost function (returns scalar)
        o
        o_
        oDim
    """

    def __init__(self, ode, cost, x0, uDim, dt, yRef,
                 terminal_cost=-1e4):
        self.tt = [0]
        self.dt = dt
        if callable(x0):
            self.x0 = x0  # initial state
            x0 = x0()
        else:
            x0 = list(x0)
            self.x0 = x0
        self.x = x0  # current state
        self.x_ = x0 # previous state x[k-1]
        self.yRef = yRef # reference values of flat output
        self.xDim = len(x0) # state dimension
        self.uDim = uDim # inputs
        self.o = self.x - np.array([*self.yRef, *np.zeros(6)])
        self.o_ = self.x_ - np.array([*self.yRef, *np.zeros(6)])
        self.oDim = len(self.o)  # observation dimensions
        self.ode = ode
        params = inspect.signature(cost).parameters
        cost_args = params.__len__()
        if cost_args == 1:
            self.cost = lambda o_, u_, o, t, mod: cost(o_)
        elif cost_args == 2:
            if 'mod' in params:
                self.cost = lambda o_, u_, o, t, mod: cost(o_, mod)
            elif 't' in params:
                self.cost = lambda o_, u_, o, t, mod: cost(o_, t)
            else:
                self.cost = lambda o_, u_, o, t, mod: cost(o_, u_)
        elif cost_args == 3:
            if 'mod' in params:
                self.cost = lambda o_, u_, o, t, mod: cost(o_, u_, mod)
            elif 't' in params:
                self.cost = lambda o_, u_, o, t, mod: cost(o_, u_, t)
            else:
                self.cost = lambda o_, u_, o, t, mod: cost(o_, u_, o)
        elif cost_args == 4:
            if 'mod' in params and 't' in params:
                self.cost = lambda o_, u_, o, t, mod: cost(o_, u_, t, mod)
            elif 'mod' in params and not 't' in params:
                self.cost = lambda o_, u_, o, t, mod: cost(o_, u_, o, mod)
            else:
                self.cost = lambda o_, u_, o, t, mod: cost(o_, u_, o, t)
        elif cost_args == 5:
            self.cost = cost
        else:
            print('Cost function must to be of the form c(o_, u_, o, t, mod), where mod is numpy/sympy.')
            assert(True)
            
        self.terminated = False
        self.terminal_cost = terminal_cost
    
    def get_state(self):
        return self.x

    def set_ref_values(self):
        '''
        yRef1 = [0.2, -0.2, 0.1, 0]
        yRef2 = [0.3, -0.3, 0.0, 0.2]
        yRef3 = [0.4, -0.1, -0.1, -0.1]
        yRef4 = [0.5, -0.4, 0.2, 0.1]
        yRefs = [yRef1, yRef2, yRef3, yRef4]
        
        selec_ind = np.random.randint(0,3)
        self.yRef = yRefs[selec_ind]
        self.yRef = self.yRef + np.random.uniform(-0.05,0.05, 4)

        '''
        self.yRef = np.random.uniform(-1.0,1.0, 4)
 
    def reset(self):
        """ Resets environment to state x0

        Args:
            x0 (array, list, callable): initial state

        """
        if callable(self.x0):
            x0 = self.x0()
        self.x_ = x0
        self.x = x0
        self.tt = [0]
        self.terminated = False
        self.set_ref_values()
        return np.array(self.x, dtype=np.float32)

    def step(self, *args):
        """ Simulates the environment for 1 step of time t.

        Args:
            dt (int, float): duration of step (not solver step size)
            u (array): control/action

        Returns:
            c (float): cost of state transition

        """
        self.x_ = self.x  # shift state (x[k-1] = x[k])
        self.o_ = self.o
        if args.__len__()==2:
            u = args[0]
            dt = args[1]
        elif args.__len__() == 1:
            u = args[0]
            dt = self.dt

        # system simulation
        sol = solve_ivp(lambda t, x: self.ode(t, x, u, self.dxdt), (0, dt), self.x_, 'RK45')
        # todo: only output value of the last timestep
        y = list(sol.y[:, -1])  # extract simulation result
        self.x = y
        self.tt.extend([self.tt[-1] + dt])  # increment simulation time
        self.terminated = self.terminate(self.x)
        #x_2pi = mapAngles(self.xIsAngle, self.x_)
        #x2pi = mapAngles(self.xIsAngle, self.x)
        #c = (self.cost(x_2pi, u, x2pi, np) + self.terminal_cost*self.terminated)*dt
        t = self.tt[-1]
        c = (self.cost(self.o_, u, self.x, t, np) + self.terminal_cost * self.terminated) * dt
        
        # todo: place difference for desired value with function or smth here
        self.o =  np.array(self.x - np.array([*self.yRef, *np.zeros(6)]), dtype=np.float32)
        reward = c
        return self.o, reward, self.terminated, {"info": False}
    
    @abstractmethod
    def terminate(self, *args):
        return



class GantryCraneEnv(StateSpaceModel):

    metadata = {}

    def __init__(self, cost, x0, dt, mod, yRef):
        super(GantryCraneEnv, self).__init__(self.ode, cost, x0, 4, dt, yRef)
        
        self.spec = gym.envs.registration.EnvSpec(id='GantryCraneEnv-v0', max_episode_steps=1000)
        
        F1, F2, F3, F4 = sp.symbols('F1 F2 F3 F4')

        params = sp.symbols('s2, m1, m2, m3, J2, l0, g')
        st.make_global(params)
        params_values = list(dict(J2=0.004553475, g=9.81, l0=0.3, m1=0.45, m2=0.557, m3=0.45, s2=0.15).items())
        
        states_dot = mod.f + mod.g * sp.Matrix([F1, F2, F3, F4]) ##:
        states_dot_wo_params = states_dot.subs(params_values)
        self.dxdt = st.expr_to_func([*mod.xx, F1, F2, F3, F4], states_dot_wo_params)

        # define observation space
        high_obs = np.array(
            [
                100.0,
                100.0,
                100.0,
                100.0,
                100.0,
                100.0, 
                100.0,
                100.0,
                100.0, 
                100.0
            ],
            dtype=np.float32,
        )
        self.observation_space = spaces.Box(-high_obs, high_obs, dtype=np.float32)
        
        # define action space
        high_act = np.array(
            [ 
                5.0,
                5.0,
                5.0,
                5.0
            ],
            dtype=np.float32,
        )
        self.action_space = spaces.Box(-high_act, high_act, dtype=np.float32)
    
    def ode(self, t, x, u, dxdt):
        u1, u2, u3, u4 = u
        states_dot = dxdt(*x, u1, u2, u3 - 2.732085, u4 - 2.732085)
        return np.array(states_dot, dtype=np.float32)
    
    def terminate(self, x):
        x1, x2, x3, x4, x5, x6, x7, x8, x9, x10 = x
        if abs(x3) > 0.5 or abs(x4) > 10 or abs(x5) > 10:
            return True
        else:
            return False

## Cost function, Initial values

In [6]:
# TODO for reference learning look on:
# https://docs.ray.io/en/latest/rllib/rllib-offline.html
# https://docs.ray.io/en/latest/rllib/package_ref/offline.html

In [7]:
mod.xx.T

Matrix([[p1, p2, p3, q1, q2, pdot1, pdot2, pdot3, qdot1, qdot2]])

In [8]:
# define the incremental cost
def c_k(o, u):
    e1, e2, e3, e4, x5, x6, x7, x8, x9, x10 = o
    u1, u2, u3, u4 = u
    
    c = -10*e1**2 -10*e2**2 - 10*e3**2 - 10*e4**2 #- x10**2 
    
    if abs(e3) < 0.02:
        c = c + 1
    
    if abs(e2) < 0.02:
        c = c + 1

    if abs(e1) < 0.02:
        c = c + 1
        
    return c

# define the function, that represents the initial value distribution p(x_0)
def p_x0():
    x0 = [
            1.49999995e-01 + np.random.uniform(-0.001,0.001), 
            -4.00000004e-01 + np.random.uniform(-0.001,0.001), 
             4.43002187e-08 + np.random.uniform(-0.001,0.001), 
            0.00000000e+00 + np.random.uniform(-0.001,0.001),
            0.00000000e+00 + np.random.uniform(-0.001,0.001),
            np.random.uniform(-0.001,0.001),
            np.random.uniform(-0.001,0.001),
            np.random.uniform(-0.001,0.001),
            np.random.uniform(-0.001,0.001),
            np.random.uniform(-0.001,0.001)
    ]
    return x0

t = 10 # time of an episode
dt = 0.01 # time step-size
yRef0 = [0.2, -0.2, 0.1, 0]

custom_local_dir="/home/kwrede/ray_results/custom_logdir"
learning_iterations = int(100) # define training iterations

## Create Gym env

In [9]:
# environment without renderings for training
env = GantryCraneEnv(c_k, p_x0, dt, mod, yRef0)

In [10]:
env.reset()
env.step(env.action_space.sample())
env.observation_space

Box([-100. -100. -100. -100. -100. -100. -100. -100. -100. -100.], [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.], (10,), float32)

## Configure RL algo

In [11]:
ray.init(dashboard_host="0.0.0.0")

2022-10-18 15:25:09,448	INFO worker.py:1518 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.0.0


In [12]:
def env_creator(env_config):
    return GantryCraneEnv(**env_config) # return an env instanceregister_env("my_env", env_creator)

register_env("GantryCraneCustom", env_creator)

## Training the model

In [13]:
import ray
import ray.rllib.algorithms.ppo as ppo
from ray.tune.logger import pretty_print

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 8
config["framework"] = "torch"
config["env_config"] = {"cost":c_k, "x0":p_x0, "dt":dt, "mod":mod, "yRef":yRef0}
config["lambda"] = 0.95
config["clip_param"] = 0.2
config["kl_coeff"] = 1.0
config["lr"] = 1e-4

algo = ppo.PPO(config=config, env="GantryCraneCustom")

2022-10-18 15:25:10,842	WARNING ppo.py:350 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=8 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 500.
2022-10-18 15:25:10,844	INFO ppo.py:378 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-10-18 15:25:10,846	INFO algorithm.py:351 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=21290) ]0;IPython: SA-Wrede/reinforcement_learning_control
(RolloutWorker pid=21289) ]0;IPython: SA-Wrede/reinforcement_learning_control
(RolloutWorker pid=21291) ]0;IPython: SA-Wrede/reinforcement_learning_control
(RolloutWorker pid=21295) ]0;IPython: SA-Wrede/reinforcement_learning_control
(RolloutWorker pid=21300) ]0;IPython: SA-Wrede/reinforcement_learning_control
(RolloutWorker pid=21297) ]0;IPython: SA-Wrede/reinforcement_learning_control
(RolloutWorker pid=21293) ]0;IPython: SA-Wrede/reinforcement_learning_control
(RolloutWorker pid=21302) ]0;IPython: SA-Wrede/reinforcement_learning_control


2022-10-18 15:25:19,706	WARNING util.py:65 -- Install gputil for GPU system monitoring.


In [24]:
for i in tqdm(range(learning_iterations)):
   # Perform one iteration of training the policy with PPO
   result = algo.train()

   if i % 100 == 0:
       checkpoint = algo.save()

  0%|▏                                                                               | 3/1000 [00:24<2:13:03,  8.01s/it]Exception ignored in: <generator object tqdm.__iter__ at 0x7f99e4609740>
Traceback (most recent call last):
  File "/home/kwrede/.local/lib/python3.8/site-packages/tqdm/std.py", line 1210, in __iter__
    self.close()
  File "/home/kwrede/.local/lib/python3.8/site-packages/tqdm/std.py", line 1316, in close
    self.display(pos=0)
  File "/home/kwrede/.local/lib/python3.8/site-packages/tqdm/std.py", line 1509, in display
    self.sp(self.__str__() if msg is None else msg)
  File "/home/kwrede/.local/lib/python3.8/site-packages/tqdm/std.py", line 350, in print_status
    fp_write('\r' + s + (' ' * max(last_len[0] - len_s, 0)))
  File "/home/kwrede/.local/lib/python3.8/site-packages/tqdm/std.py", line 344, in fp_write
    fp_flush()
  File "/home/kwrede/.local/lib/python3.8/site-packages/tqdm/utils.py", line 145, in inner
    return func(*args, **kwargs)
  File "/home/kw

KeyboardInterrupt: 

In [ ]:
algo.evaluate()

## Testing the model

In [51]:
u1_for_plots = []
u2_for_plots = []
u3_for_plots = []
u4_for_plots = []
e1_for_plots = []
e2_for_plots = []
e3_for_plots = []
e4_for_plots = []
x5_for_plots = []


obs = env.reset()
for i in range(1000):
    # inference smoothing!
    act_mean = [0, 0, 0, 0]
    for i in range(0, 4):
        action = algo.compute_single_action(obs)
        act_mean = act_mean + action
    action = act_mean/4
    obs, reward, done, info = env.step(action)
    u1_for_plots = np.append(u1_for_plots, action[0])
    u2_for_plots = np.append(u2_for_plots, action[1])
    u3_for_plots = np.append(u3_for_plots, action[2])
    u4_for_plots = np.append(u4_for_plots, action[3])
    e1_for_plots = np.append(e1_for_plots, obs[0])
    e2_for_plots = np.append(e2_for_plots, obs[1])
    e3_for_plots = np.append(e3_for_plots, obs[2])
    e4_for_plots = np.append(e4_for_plots, obs[3])
    x5_for_plots = np.append(x5_for_plots, obs[4])
    if done:
      obs = env.reset()
      break;

AttributeError: 'Tuner' object has no attribute 'compute_single_action'

## Steady state error

In [ ]:
print("e1 =", e1_for_plots[-1])
print("e2 =", e2_for_plots[-2])
print("e3 =", e3_for_plots[-3])
print("e4 =", e4_for_plots[-4])

## Plotting

In [ ]:
plt.plot(e1_for_plots, label=r"$e_1$")
plt.plot(e2_for_plots, label=r"$e_2$")
plt.plot(e3_for_plots, label=r"$e_3$")
plt.legend()

In [ ]:
plt.plot(e4_for_plots, label=r"$e_{q_1}$")
plt.plot(x5_for_plots, label=r"$q_2$")
plt.legend()

In [ ]:
plt.plot(u1_for_plots, label=r"$u_1$")
plt.plot(u2_for_plots, label=r"$u_2$")
plt.plot(u3_for_plots, label=r"$u_3$")
plt.plot(u4_for_plots, label=r"$u_4$")
plt.legend()